In [8]:
import numpy as np
#import scipy.misc
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib inline

import math

from random import shuffle

In [9]:
X = [1,2,3,4,5]
shuffle(X)
X

[1, 3, 4, 2, 5]

In [17]:
numNetworks = 100
numNodesPerNetwork = 100
inputFeatures = 10
outputFeatures = 5

globalInnovationNumber = 0

assert inputFeatures < numNodesPerNetwork
assert outputFeatures < numNodesPerNetwork

class Node:
    """
    This represents a neural network node with an integer label nodeId, 
    and a special type, one of ["input", "output", "hidden"]
    """
    def __init__(self, nodeId, nodeType):
        self.nodeId = nodeId
        assert nodeType == "input" or nodeType == "output" or nodeType == "hidden"
        self.nodeType = nodeType
        
    def __str__(self):
        return "Node: " + str(self.nodeId) + ", " + self.nodeType
        
class Edge:
    """
    This class represent a connection (axon) in a neuroevolution network with its data.
    nodeIn and nodeOut are the ids of the input and output nodes,
    enabled is a boolean representing its ability to propogate into future neural networks,
    innovId is the labeled innovation number which corresponds to the same edge in other NNs,
    and weight is the normal edge weight.
    """
    def __init__(self, nodeIn, nodeOut, enabled, innovId, weight):
        self.nodeIn = nodeIn
        self.nodeOut = nodeOut
        self.enabled = enabled
        self.innovId = innovId
        self.weight = weight
        
    def __str__(self):
        return "Edge: " + str(self.nodeIn) + " -> " + str(self.nodeOut) + ", enabled: " + str(enabled) + \
            "InnovID: " + str(self.innovId) + ", Weight: " + str(weight)
    
class NeuroevolutionNetwork:
    
    # self.nodes is a dictionary indexed by (node id -> node object)
    # self.edges is indexed by (in node id -> edge object, from in node to out node)
    def __init__(self, numNodes, numInput, numOutput):
        assert numInput + numOutput < numNodes
        
        self.nodes = dict() 
        self.edges = dict()
        self.listEdges = []
        self.numNodesCounter = 0
        
        for _ in range(numInput):
            self.addNode("input")
        for _ in range(numOutput):
            self.addNode("output")
        for _ in range(numNodes - numInput - numOutput):
            self.addNode("hidden")
        
    def copy(neuroNetwork):
        pass
        
    def addNode(self, nodeType):
        nodeId = self.numNodesCounter
        self.nodes[nodeId] = Node(nodeId, nodeType)
        self.numNodesCounter += 1
        return nodeId
        
    def hasNode(self, nodeId):
        return nodeId in self.nodes
        
    # Return true if a connection exists between nodes with these ids,
    # a connection from in -> out
    def isConnected(self, nodeInId, nodeOutId):    
        """if hasNode(nodeInId) and hasNode(nodeOutId):
            if nodeInId in self.edges:
                connectionsOut = self.edges[nodeInId]
                for nodeOut in connectionsOut:
                    if nodeOut.nodeId == nodeOutId:
                        return True
        return False"""
        return (nodeInId, nodeOutId) in self.listEdges
    
    # Return true if there is a valid connection from in -> out
    # Invariant to whether or not the nodes are connected or not
    def allowedConnection(self, nodeInId, nodeOutId):
        if hasNode(nodeInId) and hasNode(nodeOutId):
            inNode = self.nodes[nodeInId]
            outNode = self.nodes[nodeOutId]
            doubleInput = inNode.nodeType == "input" and outNode.nodeType == "output"
            doubleOutput = inNode.nodeType == "input" and outNode.nodeType == "output"
            return not doubleInput and not doubleOutput
        else:
            return False
        
    # Select a random pair of nodes, that have no connection either way,
    # and are not both input and not both output,
    # and add a connection between them, from in -> out.
    def mutateAddConnection():
        assert len(self.nodes) >= 2
        availableIds = [node.nodeId for node in self.nodes]
        availablePairs = [(availableIds[i], availableIds[j]) for i in range(availableIds) for j in range(availableIds) ]
        while True: # Find two nodes that have no connection
            if len(availablePairs) == 0:
                return
            shuffle(availablePairs)
            pair = availablePairs.pop()
            if pair[0] == pair[1]:
                continue
            inToOutC = isConnected(pair[0], pair[1])
            outToInC = isConnected(pair[1], pair[0])
            if allowedConnection(pair[0], pair[1]) and not inToOutC and not outToInC:
                newEdge = Edge(pair[0], pair[1], True, globalInnovationNumber, 0.5)
                self.listEdges.append(tuple(pair))
                globalInnovationNumber += 1
                self.edges[pair[0]].append(newEdge)
                return
           
    def removeEdge(self, nodeInId, nodeOutId):
        self.listEdges.remove((nodeInId, nodeOutId))
        
        # Look for the correct node within the outgoing connections
        for nodeOutIndex in range(len(self.edges[nodeInId])):
            nodeOut = self.edges[nodeInId][nodeOutIndex]
            if nodeOut.nodeId == nodeOutId:
                self.edges[nodeInId].pop(nodeOutIndex)
                return    
        
    """
    This mutates the graph by finding a random existing edge,
    and then splitting into two edges with a new connection in the middle,
    as per Stanley & Miikkulainen.
    """    
    def mutateSplitEdge(self):
        # Find a random edge to mutate, assuming one exists
        assert len(self.edges) > 0
        randomEdgeIndex = int(len(self.listEdges) * random.random())
        oldEdge = tuple(self.listEdges[randomEdgeIndex])
        nodeInId, nodeOutId = oldEdge[0], oldEdge[1]
        
        removeEdge(nodeInId, nodeOutId)
        
        newMiddleNode = self.addNode("hidden")
        
        newEdge1 = Edge(nodeInId, newMiddleNode, True, globalInnovationNumber, 0.5)
        self.listEdges.append(tuple(pair))
        globalInnovationNumber += 1
        self.edges[nodeInId].append(newEdge1)
        
        newEdge2 = Edge(newMiddleNode, nodeOutId, True, globalInnovationNumber, 0.5)
        self.listEdges.append(tuple(pair))
        globalInnovationNumber += 1
        self.edges[newMiddleNode].append(newEdge2)

In [23]:
# Unit tests for neuroevolution network

testNet = NeuroevolutionNetwork(numNodesPerNetwork, inputFeatures, outputFeatures)

# Nodes are initialized correctly
"""
for k,v in testNet.nodes.items():
    print(v)
"""
    
# Edges are initialized correctly
print(testNet.edges, testNet.listEdges)

{} []


In [ ]:
"""
Calculate the distance delta between two neuroevolution networks.
This heuristic is important in determining a normalized fitness score,
which rewards successful organisms but ensures that one species does not take over.

In Stanley, Miikkulainen, this is defined as

delta = c_1 * E / N + c_2 * D / N + c_3 * W
where E is the number of excess genes,
D is the number of disjoint genes, 
N is the normalization factor (number of genes total),
and W is the average weight distance between shared genes.
"""
def neuroNetworkDiff(nn1, nn2):
    pass